In [1]:
cd drive/My \Drive

/content/drive/My Drive


In [2]:
cd hackathon

/content/drive/My Drive/hackathon


In [16]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

train = pd.read_csv("train.csv")

test = pd.read_csv("test.csv")

In [17]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39161 entries, 0 to 39160
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    39161 non-null  int64  
 1   created_at            39161 non-null  object 
 2   campaign_var_1        39161 non-null  int64  
 3   campaign_var_2        39161 non-null  int64  
 4   products_purchased    18250 non-null  float64
 5   signup_date           24048 non-null  object 
 6   user_activity_var_1   39161 non-null  int64  
 7   user_activity_var_2   39161 non-null  int64  
 8   user_activity_var_3   39161 non-null  int64  
 9   user_activity_var_4   39161 non-null  int64  
 10  user_activity_var_5   39161 non-null  int64  
 11  user_activity_var_6   39161 non-null  int64  
 12  user_activity_var_7   39161 non-null  int64  
 13  user_activity_var_8   39161 non-null  int64  
 14  user_activity_var_9   39161 non-null  int64  
 15  user_activity_var_1

##First check for Na values

In [18]:
{col:f" {train[col].isna().sum()*(100/len(train))} %"  for col in train.columns}

{'buy': ' 0.0 %',
 'campaign_var_1': ' 0.0 %',
 'campaign_var_2': ' 0.0 %',
 'created_at': ' 0.0 %',
 'id': ' 0.0 %',
 'products_purchased': ' 53.39751283164372 %',
 'signup_date': ' 38.59196649728045 %',
 'user_activity_var_1': ' 0.0 %',
 'user_activity_var_10': ' 0.0 %',
 'user_activity_var_11': ' 0.0 %',
 'user_activity_var_12': ' 0.0 %',
 'user_activity_var_2': ' 0.0 %',
 'user_activity_var_3': ' 0.0 %',
 'user_activity_var_4': ' 0.0 %',
 'user_activity_var_5': ' 0.0 %',
 'user_activity_var_6': ' 0.0 %',
 'user_activity_var_7': ' 0.0 %',
 'user_activity_var_8': ' 0.0 %',
 'user_activity_var_9': ' 0.0 %'}

Output Label counts for Na values of "products_purchased" and "signup_date" features

In [84]:
train[train["products_purchased"].isna()] [["buy"]].value_counts()

buy
0      19866
1       1045
dtype: int64

In [85]:
train[train["signup_date"].isna()] [["buy"]].value_counts()

buy
0      15088
1         25
dtype: int64

In [86]:
train.describe()


,id,campaign_var_1,campaign_var_2,products_purchased,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
count,39161.000000,39161.000000,39161.000000,18250.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000,39161.000000
mean,19581.000000,6.523812,6.452746,2.154137,0.400092,0.006716,0.102832,0.011465,0.151503,0.499834,0.286612,0.174434,0.011440,0.000383,0.218942,0.000562,0.051020
std,11304.951283,3.472944,2.614296,0.779815,0.509194,0.081676,0.303743,0.106463,0.359681,0.558166,0.455784,0.379689,0.106346,0.019568,0.431544,0.023696,0.220042
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9791.000000,4.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,19581.000000,6.000000,6.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,29371.000000,9.000000,8.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,39161.000000,16.000000,15.000000,4.000000,3.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,2.000000,1.000000,1.000000,4.000000,1.000000,1.000000


##by looking at min and max values, outliers not present for any numerical features

In [20]:
from sklearn.preprocessing import MinMaxScaler,normalize

def preprocess(df,data1="test"):

  df = df.copy()

  #convert signup date  and Date of lead dropped into datetime
  for col in ["created_at","signup_date"]:
    df[col] = pd.to_datetime(df[col],dayfirst=True)
  #substract both the dates and convert it in to no of days
  df["date_diff"] =(df["created_at"] -df["signup_date"]).dt.days.abs()

  # fill na values in products_purchased and date_diff with mean value
  for col in ["products_purchased","date_diff"]:
    df[col] = df[col].fillna(df[col].mean())

  # Drop signup date  and Date of lead dropped
  for col in ["created_at","signup_date"]:
    df = df.drop(col,axis=1)
 
  if data1 == "train":
    df = df.drop("id",axis=1)
    y  = df["buy"].copy()
    x = df.drop("buy", axis=1).copy()
  else:
    x = df
    y = 0
  
  
  return x,y


x_train,y_train = preprocess(train,"train")

id_test = test["id"]
test1 = test.drop("id",axis=1).copy()
x_test,y_test = preprocess(test1)

print(x_train.shape,y_train.shape)

print(x_test.shape)


(39161, 16) (39161,)
(13184, 16)


In [14]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
clf.fit(x_train,y_train)

y_pre = clf.predict(x_test)

save = pd.DataFrame(zip(id_test,y_pre),columns =['id','buy'])

save.to_csv("sub1.csv",index=False)
print((y_pre==1).sum())
from google.colab import files
files.download("sub1.csv")

344


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>